# IdiomCheck - Language Detection



In [2]:
# -*- coding: utf-8 -*-

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import os
import glob
import numpy as np
import pandas as pd
import string

In [225]:
def corpus(file_list):
    '''
    Inputs:
    file_list - the list of paths of the text files to create the corpus
    
    Returns:
    A list of strings containing the corpus
    '''
    corpus = []
    
    for file_path in file_list:
        with open(file_path) as f_input:
            sample = re.sub('<[^>]+>', '', f_input.read()).replace('\n', ' ').replace(r"\(.*\)","").replace("(","").replace(")", "")[:10000]
            if len(sample) > 0:
                corpus.append(sample)
            
    return corpus

In [157]:
def rank(corpus, ngram_range=(1,3), rank_length=200):
    '''
    Inputs:
    corpus - a list of strings
    ngram_range - the range of ngram lengths to consider (defaults to 1-3)
    rank_length - the length of the rank list to return (defaults to 200)
    
    Retuns:
    rank - the array of ngrams ordered by frequency of occurrence in the corpus
    '''
    vectorizer = TfidfVectorizer(input='content',ngram_range=(1,3), analyzer='char_wb')
    transformed = vectorizer.fit_transform(corpus)
    features = vectorizer.get_feature_names()
    sums = transformed.sum(axis=0)
    
    return np.array(features)[np.array(np.argsort(sums[0,:]))[0]][-rank_length:]

In [57]:
def rank_sim(a, b):
    '''
    Takes two ranked arrays of the same shape and returns the average distance between the indices of matching elements
    Elements which don't match are given a distance of the length of the array
    '''
    c = a[np.where(np.in1d(a,b))[0]]
    d = c.reshape((len(c),1))
    e = np.abs(np.where(np.in1d(a,b))[0]-np.where(d==b)[1])
    no_match_penalty = (len(a)-len(e))*len(a) # TODO: make this condition more rigourous
    
    return (e.sum() + no_match_penalty)/len(a)

## Initial Test

Let's try the functions on some example data.

In [125]:
file_list_sl = glob.glob(os.path.join(os.getcwd(), "txt", "sl","*.txt"))
file_list_lv = glob.glob(os.path.join(os.getcwd(), "txt", "lv","*.txt"))

In [126]:
corpus_sl_1 = corpus(file_list_sl[:50])
corpus_sl_2 = corpus(file_list_sl[50:100])
corpus_lv_1 = corpus(file_list_lv[:50])
corpus_lv_2 = corpus(file_list_lv[50:100])

rank_sl_1 = rank(corpus_sl_1)
rank_sl_2 = rank(corpus_sl_2)
rank_lv_1 = rank(corpus_lv_1)
rank_lv_2 = rank(corpus_lv_2)

print('Slovakian/Latvian similarity score:', rank_sim(rank_sl_1, rank_lv_1))
print('Slovakian/Slovakian similarity score:', rank_sim(rank_sl_1, rank_sl_2))
print('Latvian/Latvian similarity score:', rank_sim(rank_lv_1, rank_lv_2))

Slovakian/Latvian similarity score: 127.535
Slovakian/Slovakian similarity score: 46.435
Latvian/Latvian. similarity score: 55.595


We can see that both languages show similarity to themselves (a low score means a stronger agreement between their ranks), and less similarity between each other. In this simple binary classification, the two languages would have been told apart.

## Extracting Ranks

All we will need to classify a given sample of text are the ranked ngrams for each language after analysing the whole dataset.

In [140]:
%pprint

Pretty printing has been turned OFF


In [141]:
languages = next(os.walk('./txt'))[1]
languages

['bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'it', 'lt', 'lv', 'nl', 'pl', 'pt', 'ro', 'sk', 'sl', 'sv']

In [161]:
ranks = {}

for language in languages:
    file_list = glob.glob(os.path.join(os.getcwd(), "txt", language,"*.txt"))
    lang_corpus = corpus(file_list[:50])
    lang_rank = rank(lang_corpus)
    ranks[language] = lang_rank

## Test Data

The test data comes as a .tsv file with the first column representing the language and the second column containing the string of text.

In [39]:
test = pd.read_csv('europarl-test.txt', sep='\t', header=None, names=['language', 'string'])

In [40]:
test.head()

,language,string
0,bg,Европа 2020 не трябва да стартира нов конкурен...
1,bg,(CS) Най-голямата несправедливост на сегашната...
2,bg,"(DE) Г-жо председател, г-н член на Комисията, ..."
3,bg,"(DE) Г-н председател, бих искал да започна с к..."
4,bg,"(DE) Г-н председател, въпросът за правата на ч..."


Next we need to strip the '(CS)','(DE)' etc.

In [42]:
test['string'] = test['string'].str.replace(r"\(.*\)","")
test.head()

,language,string
0,bg,Европа 2020 не трябва да стартира нов конкурен...
1,bg,Най-голямата несправедливост на сегашната общ...
2,bg,"Г-жо председател, г-н член на Комисията, по п..."
3,bg,"Г-н председател, бих искал да започна с комен..."
4,bg,"Г-н председател, въпросът за правата на човек..."


We also need to remove the rows with empty strings.

In [236]:
test.replace('', np.nan, inplace=True)
test.dropna(inplace=True)

In [237]:
test.shape

(20762, 2)

So we have a test dataset of 20828 samples. 

In [238]:
test.language.unique()

array(['bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu',
       'it', 'lt', 'lv', 'nl', 'pl', 'pt', 'ro', 'sk', 'sl', 'sv'],
      dtype=object)

In [239]:
set(test.language.unique()).issubset(languages)

True

Therefore all of the languages in the test dataset are in the training dataset, this is good.

## Classifier

In [222]:
def idiom_check(lang_sample, ranks, print_scores=False):
    '''
    Inputs:
    lang_sample - the string to be classified
    ranks - the dict() containing the language keys and their corresponding ranks
    print_scores - bool - if True, the scores for each separate language is printed
    
    Returns:
    The predicted language of lang_sample
    '''
    scores = {}
    
    sample_rank = rank([lang_sample])
    
    for key, rk in ranks.items():
        scores[key] = rank_sim(sample_rank, rk)
        
    if print_scores:
        print(scores)

    return min(scores, key=scores.get)

In [313]:
results = test.copy()[:10000]
results['pred_lang'] = results['string'].apply(lambda x: idiom_check(x, ranks))

In [314]:
print('Accuracy:',(results['language'] == results['pred_lang']).sum()/len(results))

Accuracy: 0.8344


Could also try a markov model at the character level and compare the likelihood for each sentence to be from a particular language.

## Markov Chain MLE

I aim to treat each string as a first-order Markov chain and then extract the character-level transition matrix for each language. Then I can calculate the $\log(probability)$ for a particular string to belong to each language and choose the maximally likely option.

There don't seem to be any Python packages for markov chains(?!) I will have to code this myself.

In [299]:
test = 'A pangram, or holoalphabetic sentence, is a sentence that contains every letter of the alphabet at least once. The most famous pangram is probably the thirty-five-letter-long The quick brown fox jumps over the lazy dog which has been used to test typing equipment since at least the late'
test = np.array(list(test))
test2 = 'Jelutong is a suburb of George Town in Penang, Malaysia. Located south of the Pinang River, Jelutong has been inhabited since as early as the late 18th century, when traders from Aceh and India settled around the area.'
test2 = np.array(list(test2))

In [14]:
# From - stackoverflow.com/a/43413801

def strided_axis0(a, L):
    # Store the shape and strides info
    shp = a.shape
    s  = a.strides

    # Compute length of output array along the first axis
    nd0 = shp[0]-L+1

    # Setup shape and strides for use with np.lib.stride_tricks.as_strided
    # and get (n+1) dim output array
    shp_in = (nd0,L)+shp[1:]
    strd_in = (s[0],) + s
    return np.lib.stride_tricks.as_strided(a, shape=shp_in, strides=strd_in)

This gives us the transitions as required.

In [29]:
pairs = strided_axis0(test, 2)
pairs[:10]

array([['A', ' '],
       [' ', 'p'],
       ['p', 'a'],
       ['a', 'n'],
       ['n', 'g'],
       ['g', 'r'],
       ['r', 'a'],
       ['a', 'm'],
       ['m', ','],
       [',', ' ']], dtype='<U1')

In [27]:
len(strided_axis0(test, 2))

286

In [24]:
len(np.unique(strided_axis0(test, 2), axis=0))

141

We can see that there are only 141 unique transitions out of 286 measured transitions - this is how we will build our transition matrix.

Our transition matrix could end up being quite large. We already have over 16,000 entries for the 128 characters in US-ASCII, if we extend this to the Greek characters then this could be around 4,000,000 entries.

It's probably best to store these as a (vectorised) numpy array.

In [119]:
def int_encode(groups):
    '''
    Input:
    groups - array - groups of characters to be encoded
    
    Returns:
    int_encoded - array - the integer encoded groups
    vocab - array - an array where the index of each item corresponds to the integers in int_encoded: a lookup
    
    TODO: predefined vocab as input
    '''
    vocab = np.unique(pairs)
    mask = (pairs.flatten().reshape(pairs.flatten().shape[0],1) == vocab)
    onehot_encoded = np.zeros(mask.shape)
    onehot_encoded[mask] = 1
    int_encoded = np.argmax(encoded, axis=1).reshape(pairs.shape)
    
    return int_encoded, vocab

We need to work out a way of dealing with the zero-valued probabilities - these are events that aren't need in the training data, but could be seen in the test data. For this it seems we should use some kind of [smoothing](https://pdfs.semanticscholar.org/5b2b/78087e51641a02966d6dcf20b51a5c43ccca.pdf). I'm going to use _absolute discounting_ as it is easy to implement and apparently quite effective.

In [282]:
def smooth(count_matrix):
    '''
    Implements absolute smoothing on a sparse count matrix
    
    TODO: fix this
    
    '''
    count_matrix += np.random.rand(count_matrix.shape[0],count_matrix.shape[1])/100
    
    return count_matrix

In [284]:
def transition_matrix(text):
    '''
    Input:
    text - string - the text to analyse
    
    Returns:
    trans_mat - array - the Markovian transition matrix for the text given
    vocab - array - an array where the index of each item corresponds to the position in trans_mat: a lookup
    '''
    text = np.array(list(text)) # prepare text as array of separate characters
    pairs = strided_axis0(text, 2) # window characters into consecutive pairs
    int_pairs, vocab = int_encode(pairs) # integer encode the characters
    unique, counts = np.unique(int_pairs, return_counts=True, axis=0) # count the separate instances of the transitions
    trans_mat = np.zeros((vocab.shape[0],vocab.shape[0]))
    trans_mat[unique[:,0],unique[:,1]] = counts # populate the transition matrix
    trans_mat = smooth(trans_mat)
    trans_mat = trans_mat/trans_mat.sum(axis=1).reshape(trans_mat.shape[0],1) # normalise the transition matrix (row stochastic)
    
    return trans_mat, vocab

In [296]:
def log_like(text, transition_matrix, vocab):
    '''
    Inputs:
    text - string - the sample to be analysed
    transition_matrix - array - the transition matrix containing the transition probabilities for the calculation
    vocab - array - the characters corresponding to the transition matrix entries
    
    Returns:
    log_likelihood - float - the log-likelihood for the string
    
    '''
    text = np.array(list(text))
    pairs = strided_axis0(text, 2)
    int_pairs, vocab = int_encode(pairs)
    probs = transition_matrix[int_pairs[:,0],int_pairs[:,1]]
    
    return np.log(probs).sum()

### TODO

- Feed vocab into integer encode
- Absolute Discounting